# VDJ analysis

This workbook examines T cell receptor diversity using [scirpy](https://github.com/icbi-lab/scirpy). It is heavily based on the [scirpy tutorial](https://icbi-lab.github.io/scirpy/latest/tutorials/tutorial_3k_tcr.html). For more information on the used terms, please see scirpy provided [glossary](https://icbi-lab.github.io/scirpy/latest/glossary.html). 

In [ ]:
%load_ext autoreload
%autoreload 2
import anndata

anndata.logging.anndata_logger.addFilter(
    lambda r: not r.getMessage().startswith("storing")
    and r.getMessage().endswith("as categorical.")
)


In [ ]:
import numpy as np
import os
import besca as bc
import seaborn as sns
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')

In [ ]:
import scirpy as ir
import scanpy as sc
from glob import glob
import pandas as pd
import tarfile
import anndata
import warnings

sc.set_figure_params(figsize=(4, 4))
sc.settings.verbosity = 2  # verbosity: errors (0), warnings (1), info (2), hints (3)


In [ ]:
analysis_name='sw_besca_24_final'

root_path = os.getcwd()
# Load the associated transcriptomics data

results_folder = os.path.join(root_path, 'analyzed')
results_folder = os.path.join(results_folder, analysis_name, 'citeseq' , 'citeseq')

### Create export file and folder names
results_file = os.path.join(results_folder, analysis_name + '.annotated.h5ad')
results_file = 'analyzed/'+analysis_name+'/'+ analysis_name + '.h5ad'
adata=sc.read(results_file)
figdir=os.path.join(root_path, 'analyzed', analysis_name+'/figures/')
sc.settings.figdir = figdir
if not os.path.exists(figdir):
    os.makedirs(figdir)

#### Add the annotation from individual analysis

In [ ]:
results_folder_tu=os.path.join(root_path,'analyzed',analysis_name,'citeseq' , 'citeseq')
results_folder_velo=os.path.join(root_path ,'analyzed',analysis_name)

results_file_tu = os.path.join(results_folder_tu, analysis_name + '.annotated.h5ad')
results_file_tu_paga = os.path.join(results_folder_velo, analysis_name + '.paga.h5ad')


In [ ]:
tudata=sc.read(results_file_tu)


In [ ]:
adata=tudata.copy()

In [ ]:
sc.pl.umap(adata, color=['leiden', 'celltype_simple'], legend_loc='on data')

In [ ]:
x='o25301_1_041-20210628_Sample_1_TCR o25301_1_042-20210628_Sample_2_TCR o25301_1_043-20210628_Sample_3_TCR o25301_1_044-20210628_Sample_4_TCR o25301_1_045-20210628_Sample_5_TCR o25301_1_046-20210628_Sample_6_TCR o25301_1_047-20210628_Sample_7_TCR o25301_1_048-20210628_Sample_8_TCR o25301_1_049-20210628_Sample_9_TCR o25301_1_050-20210628_Sample_10_TCR o25301_1_051-20210628_Sample_11_TCR o25301_1_052-20210628_Sample_12_TCR o25301_1_054-20210628_Sample_14_TCR o25301_1_055-20210628_Sample_15_TCR o25301_1_056-20210628_Sample_16_TCR o25301_1_058-20210628_Sample_18_TCR o25301_1_059-20210628_Sample_19_TCR o25301_1_060-20210628_Sample_20_TCR o25301_1_061-20210628_Sample_21_TCR o25301_1_062-20210628_Sample_22_TCR o25301_1_063-20210628_Sample_23_TCR o25301_1_064-20210628_Sample_24_TCR o25301_1_065-20210628_Sample_25_TCR o25301_1_066-20210628_Sample_26_TCR o25301_1_067-20210628_Sample_27_TCR o25301_1_068-20210628_Sample_28_TCR o25301_1_069-20210628_Sample_29_TCR o25301_1_070-20210628_Sample_30_TCR o25301_1_071-20210628_Sample_31_TCR o25301_1_072-20210628_Sample_32_TCR o25301_1_073-20210628_Sample_33_TCR o25301_1_074-20210628_Sample_34_TCR o25301_1_075-20210628_Sample_35_TCR o25301_1_076-20210628_Sample_36_TCR o25301_1_077-20210628_Sample_37_TCR o25301_1_078-20210628_Sample_38_TCR o25301_1_079-20210628_Sample_39_TCR o25301_1_080-20210628_Sample_40_TCR'

mynames=x.split(' ')
mynamesa=[y.split('-')[1].split("_TCR")[0] for y in x.split(' ')]

In [ ]:
tomap=[ 'IR_VJ_1_sequence', 'IR_VJ_2_sequence',
       'IR_VDJ_1_sequence', 'IR_VDJ_2_sequence', 'IR_VJ_1_sequence_aa',
       'IR_VJ_2_sequence_aa', 'IR_VDJ_1_sequence_aa', 'IR_VDJ_2_sequence_aa']
toinc=('productive', 'locus', 'v_call', 'd_call', 'j_call', 'c_call', 'junction', 'junction_aa', 'consensus_count', 'duplicate_count','is_cell', 'sequence','sequence_aa')
inpath='cellranger/vdj/'
j=0
adatas = []
for i in mynames:
    # Load the TCR data
    adata_tcr = ir.io.read_10x_vdj(inpath+i+"/outs/all_contig_annotations.csv")
    # Read the sequence information as well for later outputting the fasta sequences
    sadata_tcr = ir.io.read_airr(inpath+i+"/outs/airr_rearrangement.tsv", include_fields=toinc)
    for x in tomap:
        adata_tcr.obs[x]=sadata_tcr.obs[x].copy()
    subdata=adata[adata.obs['readout_id']==mynamesa[j]].copy()
    subdata.obs.index=[x.split('.')[1] for x in list(subdata.obs.index)]
    ir.pp.merge_with_ir(subdata, adata_tcr)
    subdata.obs.index=adata[adata.obs['readout_id']==mynamesa[j]].obs.index
    adatas.append(subdata)
    j=j+1

In [ ]:
adata = anndata.concat(adatas)

In [ ]:
sc.pl.umap(adata, color=["has_ir"])

In [ ]:
ir.tl.chain_qc(adata)

In [ ]:
ax = ir.pl.group_abundance(adata, groupby="receptor_subtype", target_col="tissue")

In [ ]:
ax = ir.pl.group_abundance(adata, groupby="celltype_simple", target_col="receptor_subtype",figsize=(14, 5))

In [ ]:
ax = ir.pl.group_abundance(adata, groupby="treatment_id", target_col="receptor_subtype")

In [ ]:

ax = ir.pl.group_abundance(adata, groupby="sample_id", target_col="receptor_subtype",figsize=(14, 5))

In [ ]:
ax = ir.pl.group_abundance(adata, groupby="readout_id", target_col="receptor_subtype",figsize=(14, 5))

In [ ]:
ax = ir.pl.group_abundance(adata, groupby="chain_pairing", target_col="treatment_id")

In [ ]:
print(
    "Fraction of cells with more than one pair of TCRs: {:.2f}".format(
        np.sum(
            adata.obs["chain_pairing"].isin(
                ["extra VJ", "extra VDJ", "two full chains"]
            )
        )
        / adata.n_obs
    )
)

In [ ]:
sc.pl.umap(adata, color="chain_pairing")

In [ ]:
alldata=adata.copy()

In [ ]:
#adata = adata[adata.obs["chain_pairing"] != "multichain", :].copy()

In [ ]:
adata = adata[~adata.obs["chain_pairing"].isin(["orphan VDJ", "orphan VJ"]), :].copy()

In [ ]:
adata = adata[~adata.obs["chain_pairing"].isin(["no IR"]), :].copy()

In [ ]:
ax = ir.pl.group_abundance(adata, groupby="chain_pairing", target_col="treatment_id")

## Explore clonotypes per treatment and cell population 

#### First: clonotypes based on nt-sequence identity (clonotypes aka cells with identical CDR3 sequence)

In [ ]:
# using default parameters, `ir_dist` will compute nucleotide sequence identity
ir.pp.ir_dist(adata)
ir.tl.define_clonotypes(adata, receptor_arms="all", dual_ir="primary_only")

In [ ]:
ir.tl.clonotype_network(adata, min_cells=10) ### change min_cells parameter for more stringent/lenient results

In [ ]:
il2vdata=adata[adata.obs['treatment_id']=='FAP-IL2v'].copy()
pd1data=adata[adata.obs['treatment_id']=='PD1'].copy()
pd1fapil2vdata=adata[adata.obs['treatment_id']=='FAP-IL2v_PD1'].copy()
pd1il2vdata=adata[adata.obs['treatment_id']=='PD1-IL2v'].copy()
vehdata=adata[adata.obs['treatment_id']=='Vehicle'].copy()

In [ ]:
tu=adata.copy()

In [ ]:
nai=adata[adata.obs['celltype_simple']=='naive'].copy()
res=adata[adata.obs['celltype_simple']=='resource'].copy()
effcytotox=adata[adata.obs['celltype_simple']=='effcytotox'].copy()
effmem=adata[adata.obs['celltype_simple']=='effmem'].copy()
effexh=adata[adata.obs['celltype_simple']=='effexh'].copy()
eff=adata[adata.obs['celltype_simple'].isin(['effmem','effcytotox','effexh'])].copy()

In these plots, each dot represents cells with identical receptor configurations (CDR3)

In [ ]:
ir.pl.clonotype_network(
    nai, color="treatment_id", base_size=7, label_fontsize=7, panel_size=(12, 12)
)

In [ ]:
ir.pl.clonotype_network(
    res, color="treatment_id", base_size=7, label_fontsize=7, panel_size=(12, 12)
)

In [ ]:
ir.pl.clonotype_network(
    effcytotox, color="treatment_id", base_size=7, label_fontsize=7, panel_size=(12, 12)
)

In [ ]:
ir.pl.clonotype_network(
    effmem, color="treatment_id", base_size=7, label_fontsize=7, panel_size=(12, 12)
)

In [ ]:
ir.pl.clonotype_network(
    effexh, color="treatment_id", base_size=7, label_fontsize=7, panel_size=(12, 12)
)

In [ ]:
ir.pl.clonotype_network(
    eff, color="treatment_id", base_size=7, label_fontsize=7, panel_size=(12, 12)
)

In [ ]:
ir.pl.clonotype_network(
    adata, color="celltype_simple", base_size=7, label_fontsize=6, panel_size=(12, 12)
)

#### Plot where clones most likely originate from 

* If we select better T effector clones, what % of them do we also find in other clusters, Per treatments? 
* Loop across clusters and find % present; LN and Tumor

In [ ]:
alldata=alldata[alldata.obs['celltype_simple'].isin(['naive','resource','effcytotox','effmem','effexh'])].copy()

In [ ]:
padata=adata[adata.obs['celltype_simple'].isin(['naive','resource','effcytotox','effmem','effexh'])].copy()
initialsubset='Tcells'

In [ ]:
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles
from matplotlib import pyplot as plt

In [ ]:
list(set(padata.obs['celltype_simple']))

In [ ]:
sc.set_figure_params(figsize=(4, 4))

In [ ]:
sc.pl.umap(tstdata, color='celltype_simple')

In [ ]:
sc.pl.umap(alldata[((alldata.obs['treatment_id']==treat))], color='celltype_simple')

In [ ]:
cella='effcytotox'
cellaname='effcytotox'
mlorigin={}
myfract={}
myfract10={}
mydiv={}
nmydiv={}
meanclones={}
medianclones={}
mytot={}
for treat in ['Vehicle','PD1','PD1-IL2v', 'FAP-IL2v','FAP-IL2v_PD1']:
#for treat in ['Vehicle']:    
    
    tstdata=padata[padata.obs['treatment_id']==treat].copy()
    #sdata=sc.pp.subsample(padata[padata.obs['treatment_id']==treat].copy(), n_obs=3500, random_state=0, copy=True)
    tmpdata=padata.obs.loc[padata.obs['treatment_id']==treat,['celltype_simple','clone_id','clone_id_size']].copy()
    myids=list(set(tmpdata.loc[tmpdata['celltype_simple']==cella,'clone_id']))
    #tmpdata=tmpdata.loc[tmpdata['tissue']=='pancreas',:]
    pertype={}
    for x in myids:
        pertype[x]=tmpdata.loc[tmpdata['clone_id']==x,:].value_counts('celltype_simple')
    pertype=pd.DataFrame(pertype)
    mlorigin[treat]=pd.Series(pertype.gt(0).sum(axis=1)/pertype.gt(0).sum(axis=1)[cella])
        
    nai=set(padata[((padata.obs['celltype_simple']=='naive')&(padata.obs['treatment_id']==treat))].obs['clone_id'])

    #### Plot venn overlaps 
    effcytotox=set(padata[((padata.obs['celltype_simple']=='effcytotox')&(padata.obs['treatment_id']==treat))].obs['clone_id'])
    re=set(padata[((padata.obs['celltype_simple']=='resource')&(padata.obs['treatment_id']==treat))].obs['clone_id'])
    effexh=set(padata[((padata.obs['celltype_simple']=='effexh')&(padata.obs['treatment_id']==treat))].obs['clone_id'])
    effmem=set(padata[((padata.obs['celltype_simple']=='effmem')&(padata.obs['treatment_id']==treat))].obs['clone_id'])
    eff=set(padata[((padata.obs['celltype_simple'].isin(['effcytotox','effmem','effexh']))&(padata.obs['treatment_id']==treat))].obs['clone_id'])
 
    venn3([re,effcytotox,effexh], set_labels = ('Resource', 'Eff/Cytotox', 'Eff/Exh'), alpha = 0.5,
          set_colors=('slategrey', 'skyblue','purple'));
    plt.title(treat)
    plt.savefig(figdir+'TCRoverlaps_Res_EffcytotoxEffexh_'+treat+'_celltype_simple.pdf')
    plt.show()
 
    venn3([re,effcytotox,effmem], set_labels = ('Resource', 'Eff/Cytotox', 'Eff/Mem'), alpha = 0.5,
          set_colors=('slategrey', 'skyblue','purple'));
    plt.title(treat)
    plt.savefig(figdir+'TCRoverlaps_Res_EffcytotoxEffmem_'+treat+'_celltype_simple.pdf')
    plt.show()

    venn3([re,effmem,effexh], set_labels = ('Resource', 'Eff/Mem', 'Eff/Exh'), alpha = 0.5,
          set_colors=('slategrey', 'skyblue','purple'));
    plt.title(treat)
    plt.savefig(figdir+'TCRoverlaps_Res_EffmemEffexh_'+treat+'_celltype_simple.pdf')
    plt.show()

    venn2([re,eff], set_labels = ('Resource', 'Effector'), alpha = 0.5,
          set_colors=('slategrey', 'skyblue'));
    plt.title(treat)
    plt.savefig(figdir+'TCRoverlaps_Res_Eff_'+treat+'_celltype_simple.pdf')
    plt.show()

    venn2([nai,eff], set_labels = ('Naive', 'Effector'), alpha = 0.5,
          set_colors=('slategrey', 'skyblue'));
    plt.title(treat)
    plt.savefig(figdir+'TCRoverlaps_Nai_Eff_'+treat+'_celltype_simple.pdf')
    plt.show()

    resonly=re-eff
    all=re.intersection(effcytotox).intersection(effmem).intersection(effexh)
    a=re.intersection(effcytotox)-all
    b=re.intersection(effmem)-all
    c=re.intersection(effexh)-all
    ab=a.intersection(b)
    ac=a.intersection(c)
    bc=b.intersection(c)
    a=a-ab-ac
    b=b-ab-bc
    c=c-ac-bc

    data=[len(resonly),len(a)+len(ab),len(b),len(c)+len(bc),len(ac),len(all)]
    labels = ['noeff', 'cytotox|cytotox&mem', 'mem only', 'exh|exh & mem',  'cytotoxexh', 'alleff']

    #define data

    #define Seaborn color palette to use
    colors = sns.color_palette('pastel')[0:7]

    #create pie chart
    plt.title(treat)
    plt.pie(data, labels = labels, autopct='%.0f%%')
    plt.savefig(figdir+'Pie_TCRoverlaps_'+treat+'_celltype_simple.pdf')
    plt.show()

    #create pie chart
    plt.title(treat)
    plt.pie(data[1:7], labels = labels[1:7], autopct='%.0f%%')
    plt.savefig(figdir+'Pie_TCRoverlaps_shared_'+treat+'_celltype_simple.pdf')
    plt.show()

    oi=list(re.intersection(eff))
    noi=list(eff-re)
    
    ntstdata=tstdata[tstdata.obs['clone_id'].isin(noi)].copy()
    ntstdata=ntstdata[ntstdata.obs['celltype_simple'].isin(['effmem','effexh','effcytotox'])].copy()
    
    tstdata=tstdata[tstdata.obs['clone_id'].isin(oi)].copy()
    tstdata=tstdata[tstdata.obs['celltype_simple'].isin(['effmem','effexh','effcytotox'])].copy()
  

    clonalitygen=tmpdata.value_counts('clone_id')
    clonality=tmpdata.loc[(tmpdata['clone_id'].isin(oi)&tmpdata['celltype_simple'].isin(['effmem','effexh','effcytotox'])),:].value_counts('clone_id')
    clonality=clonality[oi]
    plt.title(treat)
    np.log1p(clonality).hist(bins=15, density=True)
    plt.xlabel("log1p(clonality)")
    plt.savefig(figdir+'Hist_TCRclonality_shared_'+treat+'_celltype_simple.pdf')
    plt.show()
    
    reclonal=re.intersection(clonalitygen[clonalitygen>10].index)
    renonclonal=re.intersection(clonalitygen[clonalitygen<=10].index)
    effclonal=eff.intersection(clonalitygen[clonalitygen>10].index)
    effnonclonal=eff.intersection(clonalitygen[clonalitygen<=10].index)

    venn2([reclonal,effclonal], set_labels = ('Resource', 'Effector'), alpha = 0.5,
          set_colors=('slategrey', 'skyblue'));
    plt.title(treat)
    plt.savefig(figdir+'TCRoverlaps_Res_Eff_'+treat+'_celltype_simple_clonalOnly.pdf')
    plt.show()
    
    effc=len(padata[((padata.obs['celltype_simple'].isin(['effmem','effexh','effcytotox']))&(padata.obs['treatment_id']==treat)&(padata.obs['clone_id'].isin(effclonal-reclonal)))])
    rec=len(padata[((padata.obs['celltype_simple'].isin(['resource']))&(padata.obs['treatment_id']==treat)&(padata.obs['clone_id'].isin(reclonal-effclonal)))])
    receffc=len(padata[((padata.obs['celltype_simple'].isin(['resource']))&(padata.obs['treatment_id']==treat)&(padata.obs['clone_id'].isin(reclonal.intersection(effclonal))))])
    
    lenclonal=len(padata[((padata.obs['celltype_simple'].isin(['effmem','effexh','effcytotox']))&(padata.obs['treatment_id']==treat)&(padata.obs['clone_id'].isin(effclonal)))])
    lennonclonal=len(padata[((padata.obs['celltype_simple'].isin(['effmem','effexh','effcytotox']))&(padata.obs['treatment_id']==treat)&(padata.obs['clone_id'].isin(effnonclonal)))])

    lenreclonal=len(padata[((padata.obs['celltype_simple'].isin(['resource']))&(padata.obs['treatment_id']==treat)&(padata.obs['clone_id'].isin(reclonal)))])
    lenrenonclonal=len(padata[((padata.obs['celltype_simple'].isin(['resource']))&(padata.obs['treatment_id']==treat)&(padata.obs['clone_id'].isin(renonclonal)))])

    tota={}
    tota['Total_interest']=len(padata[((padata.obs['celltype_simple'].isin(['effcytotox','effmem','effexh','resource']))&(padata.obs['treatment_id']==treat))].obs['clone_id'])
    tota['Total_all']=len(padata[((padata.obs['treatment_id']==treat))].obs['clone_id'])
    tota['Total_adata']=len(alldata[((alldata.obs['treatment_id']==treat))].obs['celltype_simple'])
    tota['Total_tcrs']=len(set(padata[((padata.obs['treatment_id']==treat))].obs['clone_id']))
    tota['ClonalReNorm']=int(rec/tota['Total_adata']*10000)
    tota['ClonalReEffNorm']=int(receffc/tota['Total_adata']*10000)
    tota['ClonalEffNorm']=int(effc/tota['Total_adata']*10000)
    tota['FractionClonalEff']=lenclonal/(lenclonal+lennonclonal)
    tota['FractionClonalRes']=lenreclonal/(lenreclonal+lenrenonclonal)
    mytot[treat]=tota
    
    venn2(subsets=(int(len(re-eff)/tota['Total_adata']*10000),int(len(eff-re)/tota['Total_adata']*10000),int(len(eff.intersection(re))/tota['Total_adata']*10000)), set_labels = ('Resource', 'Effector'))
    plt.title(treat)
    plt.savefig(figdir+'TCRoverlaps_Res_Eff_'+treat+'_celltype_simple_relativeTo1k.pdf')
    plt.show()
    
    
    venn2(subsets=(int(len(reclonal-effclonal)/tota['Total_adata']*10000),int(len(effclonal-reclonal)/tota['Total_adata']*10000),int(len(effclonal.intersection(reclonal))/tota['Total_adata']*10000)), set_labels = ('Resource', 'Effector'))
    plt.title(treat)
    plt.savefig(figdir+'TCRoverlaps_Res_Eff_'+treat+'_celltype_simple_clonalOnly_relativeTo1k.pdf')
    plt.show()
    
    venn2(subsets = (tota['ClonalReNorm'],  tota['ClonalEffNorm'],tota['ClonalReEffNorm']), set_labels = ('Resource', 'Effector'))
    plt.title(treat)
    plt.savefig(figdir+'TCRoverlaps_Res_Eff_'+treat+'_celltype_simple_cellnrs_clonalOnly_relativeTo1k.pdf')
    plt.show()
    ### correct for total cell nrs. 
    #clonality=clonality*5000/len(tst)
    
    #len(clonality[clonality==1])
    data=[len(clonality[clonality<=2]),len(clonality[(clonality>2)&(clonality<10)]),
    len(clonality[(clonality>=10)&(clonality<100)]),len(clonality[(clonality>=100)])]
    labels=['1','2-9','10-100','>100']
    #define Seaborn color palette to use
    colors = sns.color_palette('pastel')[0:7]

    ir.tl.alpha_diversity(ntstdata, groupby='sample_id', key_added='alpha_diversity_ind')

    ir.tl.alpha_diversity(tstdata, groupby='sample_id', key_added='alpha_diversity_ind')
    
    #create pie chart
    plt.title(treat)
    plt.pie(data, labels = labels, autopct='%.0f%%', colors=colors)
    #plt.savefig(figdir+'Pie_TCRoverlaps_'+treat+'_celltype_simple.pdf')
    plt.show()
    plt.savefig(figdir+'Pie_TCRclonality_shared_'+treat+'_celltype_simple.pdf')

    eff10=set(list(clonality[clonality>=10].index))
    
    myfract[treat]=len(list(re.intersection(eff)))/len(list(re))
    myfract10[treat]=len(list(re.intersection(eff10)))/len(list(re))
    mydiv[treat]=tstdata.obs.loc[:,['alpha_diversity_ind','individual_id', 'treatment_id']].drop_duplicates()
    nmydiv[treat]=ntstdata.obs.loc[:,['alpha_diversity_ind','individual_id', 'treatment_id']].drop_duplicates()
    meanclones[treat]=pd.DataFrame(np.log1p(tstdata.obs.value_counts(['clone_id','individual_id'])).groupby('individual_id').mean())
    meanclones[treat]['treatment_id']=treat
    medianclones[treat]=pd.DataFrame(np.log1p(tstdata.obs.value_counts(['clone_id','individual_id'])).groupby('individual_id').median())
    medianclones[treat]['treatment_id']=treat
    print(treat+': '+str(myfract))

In [ ]:
set(adata.obs['sample_id'])

In [ ]:
pd.DataFrame(mytot)

In [ ]:
effclonal=eff.intersection(clonalitygen[clonalitygen>1].index)
effnonclonal=eff.intersection(clonalitygen[clonalitygen<=1].index)

In [ ]:
efft=len(padata[((padata.obs['celltype_simple'].isin(['effmem','effexh','effcytotox']))&(padata.obs['treatment_id']==treat))])

In [ ]:
ret=len(padata[((padata.obs['celltype_simple'].isin(['resource']))&(padata.obs['treatment_id']==treat))])

In [ ]:
pd.Series([324,67,476])/9482*10000

In [ ]:
pd.Series([307,87,403])/6433*10000

In [ ]:
meanclones=pd.concat(meanclones)
meanclones.columns=['log_clone_nr','treatment_id']

In [ ]:
medianclones=pd.concat(medianclones)
medianclones.columns=['log_clone_nr','treatment_id']

In [ ]:
medianclones.to_csv(figdir+'ResourceEffshared_MedianClones_perTreatment.tsv')

In [ ]:
sns.set(rc={'figure.figsize':(5,3)})
sns.set_style('ticks')
sns.boxplot(x='log_clone_nr',y='treatment_id', data=medianclones)
sns.swarmplot(x='log_clone_nr',y='treatment_id', data=medianclones, color='black')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.savefig(figdir+'ResourceEffshared_MedianClones_perTreatment.pdf')

In [ ]:
meanclones.to_csv(figdir+'ResourceEffshared_MeanClones_perTreatment.tsv')

In [ ]:
sns.set(rc={'figure.figsize':(5,3)})
sns.set_style('ticks')
sns.boxplot(x='log_clone_nr',y='treatment_id', data=meanclones)
sns.swarmplot(x='log_clone_nr',y='treatment_id', data=meanclones, color='black')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.savefig(figdir+'ResourceEffshared_MeanClones_perTreatment.pdf')

In [ ]:
sns.set(rc={'figure.figsize':(2.5,1.5)})
sns.set_style('ticks')
sns.boxplot(x='log_clone_nr',y='treatment_id', data=medianclones.loc[medianclones['treatment_id'].isin(['PD1','PD1-IL2v','FAP-IL2v_PD1'])], order=['PD1-IL2v','FAP-IL2v_PD1','PD1'],)
sns.swarmplot(x='log_clone_nr',y='treatment_id', data=medianclones.loc[medianclones['treatment_id'].isin(['PD1','PD1-IL2v','FAP-IL2v_PD1'])], color='black', order=['PD1-IL2v','FAP-IL2v_PD1','PD1'],)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.savefig(figdir+'ResourceEffshared_MedianClones_perTreatment_3Tonly.pdf')

In [ ]:
sns.set(rc={'figure.figsize':(5,3)})
sns.set_style('ticks')
sns.boxplot(x='alpha_diversity_ind',y='treatment_id', data=pd.concat(mydiv))
sns.swarmplot(x='alpha_diversity_ind',y='treatment_id', data=pd.concat(mydiv), color='black')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.savefig(figdir+'ResourceEffshared_Alphadiveristy_perTreatment.pdf')

In [ ]:
pd.concat(mydiv).to_csv(figdir+'ResourceEffshared_Alphadiveristy_perTreatment.tsv', sep='\t')

In [ ]:
sns.set(rc={'figure.figsize':(3,2)})
sns.set_style('ticks')
sns.boxplot(x='alpha_diversity_ind',y='treatment_id', order=['PD1-IL2v','FAP-IL2v_PD1','PD1'],
            data=pd.concat(mydiv).loc[pd.concat(mydiv)['treatment_id'].isin(['PD1','PD1-IL2v','FAP-IL2v_PD1'])])
sns.swarmplot(x='alpha_diversity_ind',y='treatment_id', order=['PD1-IL2v','FAP-IL2v_PD1','PD1'],
              data=pd.concat(mydiv).loc[pd.concat(mydiv)['treatment_id'].isin(['PD1','PD1-IL2v','FAP-IL2v_PD1'])], color='black')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
#plt.xlim(0.5, 1)
plt.savefig(figdir+'ResourceEffshared_Alphadiveristy_perTreatment_3Tonly.pdf')

In [ ]:
sns.set(rc={'figure.figsize':(3,2)})
sns.set_style('ticks')
sns.boxplot(x='alpha_diversity_ind',y='treatment_id', order=['PD1-IL2v','FAP-IL2v_PD1','PD1'],
            data=pd.concat(nmydiv).loc[pd.concat(nmydiv)['treatment_id'].isin(['PD1','PD1-IL2v','FAP-IL2v_PD1'])])
sns.swarmplot(x='alpha_diversity_ind',y='treatment_id', order=['PD1-IL2v','FAP-IL2v_PD1','PD1'],
              data=pd.concat(nmydiv).loc[pd.concat(nmydiv)['treatment_id'].isin(['PD1','PD1-IL2v','FAP-IL2v_PD1'])], color='black')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xlim(0.5, 1)
plt.savefig(figdir+'Effspec_Alphadiveristy_perTreatment_3Tonly.pdf')

In [ ]:

myfract=pd.DataFrame([myfract,myfract,myfract10]).transpose()
myfract10=pd.DataFrame([myfract10,myfract10]).transpose()

In [ ]:
myfract.columns=['res only','shared <10','shared >=10']
myfract['res only']=1-myfract['res only']
myfract['shared <10']=myfract['shared <10']-myfract['shared >=10']

myfract10.columns=['res','shared']
myfract10['res']=1-myfract10['res']

In [ ]:
myfract=myfract.loc[['PD1-IL2v','PD1','FAP-IL2v_PD1','Vehicle','FAP-IL2v'],:]
myfract10=myfract10.loc[['PD1-IL2v','PD1','FAP-IL2v_PD1','Vehicle','FAP-IL2v'],:]

In [ ]:
myfract.plot.barh(stacked=True, color={'res only': 'gray', 'shared <10': 'orange', 'shared >=10': 'red'})
plt.savefig(figdir+'Resource_TCRclonality_shared_with_resource_celltype_simple.pdf')

In [ ]:
myfract.to_csv(figdir+'Resource_TCRclonality_shared_with_resource_celltype_simple.tsv', sep='\t')

In [ ]:
ir.tl.alpha_diversity(adata, groupby='IND_lei', key_added='alpha_diversity_ind_leiden')

In [ ]:
#cella='better T effector CD8-positive, alpha-beta T cell'
cella='effcytotox'
cellb='resource'
binaries={}
for treat in ['Vehicle','PD1','PD1-IL2v', 'FAP-IL2v','FAP-IL2v_PD1']:
    tmpdata=padata.obs.loc[padata.obs['treatment_id']==treat,['celltype_simple','clone_id','clone_id_size']].copy()
    tmpdata=tmpdata.loc[tmpdata['celltype_simple'].isin([cella,cellb]),:]
    tmpplot=pd.DataFrame(tmpdata.value_counts(['clone_id','celltype_simple']))
    tmpplot.reset_index(inplace=True)
    tmpplot.index=list(tmpplot['clone_id'])
    tmpplot=tmpplot.drop(columns='clone_id')
    tmpplot.columns=['celltype_simple','nr_cells']
    allvals={}
    for x in list(tmpplot.index):
        sub=tmpplot.loc[x,:].copy()
        if len(sub['celltype_simple'])==2:
            sub=sub.reset_index().drop(columns='index')
            a=sub.loc[sub['celltype_simple']==cella,'nr_cells']
            b=sub.loc[sub['celltype_simple']==cellb,'nr_cells']
            myvals=list(a)+list(b)
        else:
            if (sub['celltype_simple']==cellb):
                b=[sub['nr_cells']]
                myvals=[0]+b
            else:
                a=[sub['nr_cells']]
                myvals=a+[0]
        allvals[x]=myvals
    allvals=np.log1p(pd.DataFrame(allvals).transpose())
    allvals.columns=[cella,cellb]
    allvals['color']='shared'
    allvals.loc[(allvals[cella]==0),'color']='uniquea'
    allvals.loc[(allvals[cellb]==0),'color']='uniqueb'
    binaries[treat]=allvals.copy()

In [ ]:
for key in binaries.keys():
#    g=np.log1p(binaries[key]).plot(cella, cellb, kind='scatter')
#    plt.title(key)
#    plt.xlim([-0.5, 7])
#    plt.ylim([-0.5, 7])
#    plt.savefig(figdir+cellaname+'_TCRscattterplots_TermExh_Resource'+key+'.pdf')
#plt.savefig(figdir+cellaname+'_TCRscattterplots_BetterTeff_Resource.pdf')
    plt.figure()
    sns.set(style='white')
    sns.set_context("paper", font_scale=1.2) 
    sns.scatterplot(x=cella,y=cellb,hue='color',palette='deep',data=binaries[key])

    plt.legend([],[], frameon=False)
    plt.title(key)
    plt.xlim([-0.5, 7])
    plt.ylim([-0.5, 7])
    plt.savefig(figdir+cellaname+'_TCRscattterplots_EffCytotox_Resource'+key+'.pdf')

In [ ]:
for key in binaries.keys():
    Axes=pd.plotting.scatter_matrix(binaries[key], alpha=0.1,diagonal='kde')
    plt.title(key)
    [plt.setp(item.yaxis.get_label(), 'size', 6) for item in Axes.ravel()]
    [plt.setp(item.xaxis.get_label(), 'size', 6) for item in Axes.ravel()]
    #plt.savefig(figdir+cellaname+'_TCRscattterplots_BetterTeff_Resource.pdf')
    plt.savefig(figdir+cellaname+'_TCRscattterplots_EffCytotox_Resource_kde_'+key+'.pdf')



In [ ]:
adata.write('analyzed/sw_besca_24_final/sw_besca_24_final-vdj.h5ad')

#### Clonotype network analysis

In [ ]:
### Perform only on a subset of data
#subdata=adata[adata.obs['leiden_sep'].isin(['Tu6','Tu13','Tu5','Tu4','Tu1','Tu2','Tu3','Tu0','Tu9','Tu11', 'Tu11','Tu8','Tu15'])].copy()
subdata=subdata[subdata.obs['treatment_id'].isin(['Vehicle','PD1','PD1-IL2v', 'FAP-IL2v','FAP-IL2v_PD1'])].copy()

In [ ]:
subdata=tu.copy()

In [ ]:
ir.pp.ir_dist(
    subdata,
    metric="alignment",
    sequence="aa",
    cutoff=15,
)

In [ ]:
ir.tl.define_clonotype_clusters(
    subdata, sequence="aa", metric="alignment", receptor_arms="all", dual_ir="any"
)

In [ ]:
ir.tl.clonotype_network(subdata, min_cells=10, sequence="aa", metric="alignment")

In [ ]:
subdata.write('analyzed/sw_besca_24_final/sw_besca_24_final-subdata-vdj.h5ad')

In [ ]:
ir.tl.define_clonotype_clusters(
    subdata,
    sequence="aa",
    metric="alignment",
    receptor_arms="all",
    dual_ir="primary_only",
    same_v_gene=True,
    key_added="cc_aa_alignment_same_v",
)

In [ ]:
# find clonotypes with more than one `clonotype_same_v`
ct_different_v = subdata.obs.groupby("cc_aa_alignment").apply(
    lambda x: x["cc_aa_alignment_same_v"].nunique() > 1
)
ct_different_v = ct_different_v[ct_different_v].index.values.tolist()
len(ct_different_v)

In [ ]:
subdata.obs.loc[
    subdata.obs["cc_aa_alignment"].isin(ct_different_v),
    [
        "cc_aa_alignment",
        "cc_aa_alignment_same_v",
        "IR_VJ_1_v_call",
        "IR_VDJ_1_v_call",
    ],
].sort_values("cc_aa_alignment").drop_duplicates().reset_index(drop=True)

In [ ]:
spadata=subdata[subdata.obs['celltype_simple'].isin(['naive','resource','effcytotox','effmem','effexh'])].copy()
initialsubset='Tcells'

In [ ]:
treats=['Vehicle','PD1','PD1-IL2v', 'FAP-IL2v','FAP-IL2v_PD1']

In [ ]:
tmpdata=spadata.obs.loc[spadata.obs['treatment_id']==treat,['treatment_id','clone_id','clone_id_size']].copy()

In [ ]:
binaries={}
for i in range(0,len(treats)):
    for j in range(i+1, len(treats)):
        if treats[i]!=treats[j]:
            #cella='better T effector CD8-positive, alpha-beta T cell'
            cella=treats[i]
            cellb=treats[j]
            #for treat in ['Vehicle','PD1','PD1-IL2v', 'FAP-IL2v','FAP-IL2v_PD1']:
            tmpdata=spadata.obs.loc[:,['treatment_id','clone_id','clone_id_size']].copy()
            tmpdata=tmpdata.loc[tmpdata['treatment_id'].isin([cella,cellb]),:]
            tmpplot=pd.DataFrame(tmpdata.value_counts(['clone_id','treatment_id']))
            tmpplot.reset_index(inplace=True)
            tmpplot.index=list(tmpplot['clone_id'])
            tmpplot=tmpplot.drop(columns='clone_id')
            tmpplot.columns=['treatment_id','nr_cells']
            allvals={}
            for x in list(tmpplot.index):
                sub=tmpplot.loc[x,:].copy()
                if len(sub['treatment_id'])==2:
                    sub=sub.reset_index().drop(columns='index')
                    a=sub.loc[sub['treatment_id']==cella,'nr_cells']
                    b=sub.loc[sub['treatment_id']==cellb,'nr_cells']
                    myvals=list(a)+list(b)
                else:
                    if (sub['treatment_id']==cellb):
                        b=[sub['nr_cells']]
                        myvals=[0]+b
                    else:
                        a=[sub['nr_cells']]
                        myvals=a+[0]
                allvals[x]=myvals
            allvals=np.log1p(pd.DataFrame(allvals).transpose())
            allvals.columns=[cella,cellb]
            allvals['color']='shared'
            allvals.loc[(allvals[cella]==0),'color']='uniquea'
            allvals.loc[(allvals[cellb]==0),'color']='uniqueb'
            binaries[cella+'-vs-'+cellb]=allvals

In [ ]:
compsums={}
for key in binaries.keys():
    plt.figure()
    sns.set(style='white')
    sns.set_context("paper", font_scale=1.2) 
    sns.scatterplot(x=key.split('-vs-')[0],y=key.split('-vs-')[1],
                    hue='color',palette='deep',data=binaries[key])

    plt.legend([],[], frameon=False)
    plt.title(key)
    plt.xlim([-0.5, 7])
    plt.ylim([-0.5, 7])
    plt.savefig(figdir+'AllTcells_TCRscattterplots_Treatments_TuOnly_'+key+'.pdf')
    
    nonzero=binaries[key].loc[(binaries[key][key.split('-vs-')[0]]>0)&(binaries[key][key.split('-vs-')[1]]>0),:]
    print(nonzero)
    compsums[key]=len(nonzero)/len(binaries[key])


In [ ]:
pd.Series(compsums).sort_values(ascending=False).plot(kind='bar')
plt.ylabel("Fraction overlapping clones")
plt.savefig(figdir+'AllTcells_TCRscattterplots_subdata_'+key+'.pdf')

#### Now based on aa similarity 

In [ ]:
tmpdata=spadata.obs.loc[spadata.obs['treatment_id']==treat,['treatment_id','cc_aa_alignment','cc_aa_alignment_size']].copy()

binaries={}
for i in range(0,len(treats)):
    for j in range(i+1, len(treats)):
        if treats[i]!=treats[j]:
            #cella='better T effector CD8-positive, alpha-beta T cell'
            cella=treats[i]
            cellb=treats[j]
            #for treat in ['Vehicle','PD1','PD1-IL2v', 'FAP-IL2v','FAP-IL2v_PD1']:
            tmpdata=spadata.obs.loc[:,['treatment_id','cc_aa_alignment','cc_aa_alignment_size']].copy()
            tmpdata=tmpdata.loc[tmpdata['treatment_id'].isin([cella,cellb]),:]
            tmpplot=pd.DataFrame(tmpdata.value_counts(['cc_aa_alignment','treatment_id']))
            tmpplot.reset_index(inplace=True)
            tmpplot.index=list(tmpplot['cc_aa_alignment'])
            tmpplot=tmpplot.drop(columns='cc_aa_alignment')
            tmpplot.columns=['treatment_id','nr_cells']
            allvals={}
            for x in list(tmpplot.index):
                sub=tmpplot.loc[x,:].copy()
                if len(sub['treatment_id'])==2:
                    sub=sub.reset_index().drop(columns='index')
                    a=sub.loc[sub['treatment_id']==cella,'nr_cells']
                    b=sub.loc[sub['treatment_id']==cellb,'nr_cells']
                    myvals=list(a)+list(b)
                else:
                    if (sub['treatment_id']==cellb):
                        b=[sub['nr_cells']]
                        myvals=[0]+b
                    else:
                        a=[sub['nr_cells']]
                        myvals=a+[0]
                allvals[x]=myvals
            allvals=np.log1p(pd.DataFrame(allvals).transpose())
            allvals.columns=[cella,cellb]
            allvals['color']='shared'
            allvals.loc[(allvals[cella]==0),'color']='uniquea'
            allvals.loc[(allvals[cellb]==0),'color']='uniqueb'
            binaries[cella+'-vs-'+cellb]=allvals



In [ ]:
compsums={}
for key in binaries.keys():
    plt.figure()
    sns.set(style='white')
    sns.set_context("paper", font_scale=1.2) 
    sns.scatterplot(x=key.split('-vs-')[0],y=key.split('-vs-')[1],
                    hue='color',palette='deep',data=binaries[key])

    plt.legend([],[], frameon=False)
    plt.title(key)
    plt.xlim([-0.5, 7])
    plt.ylim([-0.5, 7])
    plt.savefig(figdir+'AllTcells_TCRscattterplots_Treatments_TuOnly_AAsimilarity'+key+'.pdf')
    
    nonzero=binaries[key].loc[(binaries[key][key.split('-vs-')[0]]>0)&(binaries[key][key.split('-vs-')[1]]>0),:]
    print(nonzero)
    compsums[key]=len(nonzero)/len(binaries[key])


In [ ]:
pd.Series(compsums).sort_values(ascending=False).plot(kind='bar')
plt.ylabel("Fraction overlapping clones")
plt.savefig(figdir+'AllTcells_TCRscattterplots_subdata_AAsimilarity'+key+'.pdf')

#### Clonal expansion 

In [ ]:
ir.tl.clonal_expansion(adata)

In [ ]:
sc.set_figure_params(figsize=(4, 4))

In [ ]:
sc.pl.umap(adata, color=["clonal_expansion", "clone_id_size"])

In [ ]:
adata.obs["clone_id_size_log1p"]=list(np.log1p(adata.obs["clone_id_size"]))

In [ ]:
sc.pl.umap(adata, color=["clonal_expansion", "clone_id_size","clone_id_size_log1p"],ncols=1,
           save='Clonal_expansion_alldata.pdf')

In [ ]:
subtudata=tudata[tudata.obs.index.isin(adata.obs.index)].copy()
subtudata=subtudata[subtudata.obs['celltype_simple'].isin(['naive','resource','effcytotox','effmem','effexh'])].copy()
initialsubset='Tcells'

In [ ]:
sc.pl.umap(subtudata,color='celltype_simple')

In [ ]:
subtudata.obs['clone_id_size']=list(adata[subtudata.obs.index].obs['clone_id_size'])
subtudata.obs['clone_id_size_log1p']=list(adata[subtudata.obs.index].obs['clone_id_size_log1p'])
subtudata.obs['clonal_expansion']=list(adata[subtudata.obs.index].obs['clonal_expansion'])

In [ ]:
sc.pl.umap(subtudata, color=["clonal_expansion", "clone_id_size","clone_id_size_log1p"],
           ncols=1,save='Clonal_expansion_tudata.pdf', color_map='viridis')

In [ ]:
subtudata.obs.loc[:,['leiden','celltype','clone_id_size_log1p']]

In [ ]:
sc.pl.umap(adata, color=["clonal_expansion", "clone_id_size"])

In [ ]:
ir.pl.clonal_expansion(adata, groupby="CONDITION", clip_at=20, normalize=True,figsize=(8, 5))

In [ ]:
ir.pl.clonal_expansion(tu, groupby="CONDITION", clip_at=20, normalize=True,figsize=(5, 5))

In [ ]:
ir.pl.clonal_expansion(tu, groupby="sample_id", clip_at=20, normalize=True,figsize=(14, 5))

In [ ]:
ir.pl.clonal_expansion(adata, groupby="leiden", clip_at=10, normalize=True,figsize=(14, 5))

In [ ]:
ir.pl.clonal_expansion(tu[tu.obs['celltype_simple']!='other'], groupby="celltype_simple", clip_at=20, 
                       normalize=True,figsize=(5, 5))
plt.savefig(figdir+'ClonalExpansion_celltype_simple.pdf')

In [ ]:
adata.write('analyzed/sw_besca_24_final/sw_besca_24_final-vdj.h5ad')

In [ ]:
il2vdata=adata[adata.obs['treatment_id']=='FAP-IL2v'].copy()
pd1data=adata[adata.obs['treatment_id']=='PD1'].copy()
pd1fapil2vdata=adata[adata.obs['treatment_id']=='FAP_IL2v_PD1'].copy()
pd1il2vdata=adata[adata.obs['treatment_id']=='PD1-IL2v'].copy()
vehdata=adata[adata.obs['treatment_id']=='Vehicle'].copy()
tu=adata[adata.obs['tissue']=='pancreas'].copy()
nai=adata[adata.obs['celltype_simple']=='naive'].copy()
res=adata[adata.obs['celltype_simple']=='resource'].copy()
effcytotox=adata[adata.obs['celltype_simple']=='effcytotox'].copy()
effmem=adata[adata.obs['celltype_simple']=='effmem'].copy()
effexh=adata[adata.obs['celltype_simple']=='effexh'].copy()
eff=adata[adata.obs['celltype_simple'].isin(['effmem','effcytotox','effexh'])].copy()

In [ ]:
sc.pl.umap(adata, color='leiden', legend_loc='on data')

In [ ]:
ir.pl.clonal_expansion(adata, "CONDITION", clip_at=10)

In [ ]:
ir.pl.clonal_expansion(tu, "treatment_id", clip_at=10)

In [ ]:
ax = ir.pl.alpha_diversity(adata, groupby="leiden",figsize=(14, 5))
plt.savefig(figdir+'Alpha_diversity_leiden_adata.pdf')

In [ ]:
ax = ir.pl.alpha_diversity(adata, groupby="treatment_id",figsize=(3, 3))
plt.savefig(figdir+'Alpha_diversity_treatmen_id_adata.pdf')

In [ ]:
ax = ir.pl.alpha_diversity(tu[tu.obs['celltype_simple'].isin(['effmem','effcytotox','effexh'])], groupby="treatment_id",figsize=(3, 3))
plt.savefig(figdir+'Alpha_diversity_treatmen_id_matureCD8T.pdf')

In [ ]:
ax = ir.pl.alpha_diversity(tu[tu.obs['celltype_simple'].isin(['effmem','effcytotox','effexh','naive','resource'])], groupby="celltype_simple",figsize=(3, 3))
plt.savefig(figdir+'Alpha_diversity_celltype_simple_matureCD8T.pdf')

In [ ]:
adata.obs['IND_cell']=list(adata.obs['sample_id'].astype(str)+"_"+adata.obs['celltype_simple'].astype(str))

In [ ]:
adata.obs['IND_lei']=list(adata.obs['sample_id'].astype(str)+"_"+adata.obs['leiden'].astype(str))

In [ ]:
ir.tl.alpha_diversity(adata, groupby='IND_cell')

In [ ]:
ir.tl.alpha_diversity(adata, groupby='IND_lei', key_added='alpha_diversity_ind_leiden')

In [ ]:
adata.write('analyzed/sw_besca_24_final/sw_besca_24_final-vdj.h5ad')

In [ ]:
#sc.read('analyzed/sw_besca_24_final/sw_besca_24_final-vdj.h5ad')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
tmp=adata[adata.obs['tissue']=='pancreas'].obs.loc[:,['clone_id_size','alpha_diversity_clone_id','treatment_id','celltype_simple','sample_id']].copy()

In [ ]:
cellorder=['naive','resource','effmem','effcytotox','effexh']

In [ ]:
cellorder2=['resource','effmem','effcytotox','effexh']

In [ ]:
sub=tmp.loc[tmp['celltype_simple'].isin(cellorder),['alpha_diversity_clone_id','treatment_id','celltype_simple']].copy()
sub['celltype_sep']=sub['celltype_simple'].cat.remove_unused_categories()
sub=sub.drop_duplicates()

In [ ]:
sub2=tmp.loc[tmp['celltype_simple'].isin(cellorder2),['alpha_diversity_clone_id','treatment_id','celltype_simple']].copy()
sub2['celltype_simple']=sub2['celltype_simple'].cat.remove_unused_categories()
sub2=sub2.drop_duplicates()

In [ ]:
#sub2=np.log2(tmp.loc[tmp['celltype_sep'].isin(cellorder),:].groupby(['sample_id','celltype_sep']).mean('alpha_diversity_clone_id')).drop_duplicates()

In [ ]:
sns.set(rc={'figure.figsize':(6,14)})
sns.set_style('ticks')
sns.boxplot(x='alpha_diversity_clone_id',y='celltype_simple',hue='treatment_id', orient='h',data=sub, order=cellorder)
sns.swarmplot(x='alpha_diversity_clone_id',y='celltype_simple',
              hue='treatment_id', orient='h',data=sub, color=".25",dodge=True, order=cellorder)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
sns.set(rc={'figure.figsize':(5,6)})
sns.set_style('ticks')
sns.boxplot(x='alpha_diversity_clone_id',y='celltype_simple', data=sub, order=cellorder)
sns.swarmplot(x='alpha_diversity_clone_id',y='celltype_simple', data=sub, color=".25", order=cellorder)

In [ ]:
sns.set(rc={'figure.figsize':(5,6)})
sns.set_style('ticks')
sns.boxplot(x='alpha_diversity_clone_id',y='treatment_id', data=sub, color='lightgray')
sns.swarmplot(x='alpha_diversity_clone_id',y='treatment_id', data=sub,  hue='celltype_simple')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
sns.set(rc={'figure.figsize':(5,3)})
sns.set_style('ticks')
sns.boxplot(x='alpha_diversity_clone_id',y='treatment_id', data=sub2, color='lightgray')
sns.swarmplot(x='alpha_diversity_clone_id',y='treatment_id', hue='celltype_simple',data=sub2)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.savefig(figdir+'AlphaDiversity_per_treatment_celltype_simple.pdf')

In [ ]:
sns.set(rc={'figure.figsize':(5,3)})
sns.set_style('ticks')
sns.boxplot(x='alpha_diversity_clone_id',y='treatment_id', data=sub2.loc[sub2['treatment_id'].isin(['FAP-IL2v_PD1','PD1','PD1-IL2v']),:], color='lightgray')
sns.swarmplot(x='alpha_diversity_clone_id',y='treatment_id', hue='celltype_simple',data=sub2.loc[sub2['treatment_id'].isin(['FAP-IL2v_PD1','PD1','PD1-IL2v']),:])
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.savefig(figdir+'AlphaDiversity_per_treatment_celltype_simple_3Tonly.pdf')

In [ ]:
sub2

### Convert to html


In [ ]:

%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')




In [ ]:
nb_name = os.path.join(os.getcwd(), nb_name)

! jupyter nbconvert --to html {nb_name}